#API para utilizar el modelo de redes neuronales para detección de neumonía.

A continuación, se realizará la configuración de una API con Flask, la cual se ejecuta localmente. Para permitir el acceso desde internet, se utilizará Ngrok, que expone temporalmente el servidor local mediante una URL pública. Todos los archivos necesarios para la demo (modelo, radiografías) están alojados en Google Drive.

##Librerias

In [19]:
!pip install flask-ngrok
!pip install pyngrok
!pip install --upgrade certifi requests urllib3
!kill -9 $(lsof -t -i:5000)
!pkill -f ngrok

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [20]:
!fuser -k 5000/tcp
!ngrok authtoken 2sVs31iobFm9ntGLeNNumFTU1h4_2ZB3GuBS68cVGyY9mxigS

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from PIL import Image
import numpy as np
import io
from pyngrok import ngrok
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import shutil
import os
from werkzeug.utils import secure_filename

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
#from google.colab import files
#uploaded = files.upload()

In [6]:
os.makedirs('/content/templates', exist_ok=True)
os.makedirs('static', exist_ok=True)

In [7]:
shutil.copy('/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/static/Logo AIRE.png', 'static/logo_aire.png')

'static/logo_aire.png'

##Creación de la App Flask

In [8]:
app = Flask(__name__)
public_url = ngrok.connect(5000)
print("URL pública:", public_url)
run_with_ngrok(app)

URL pública: NgrokTunnel: "https://0b11-34-74-198-234.ngrok-free.app" -> "http://localhost:5000"


In [9]:
# Configuraciones para el archivo cargado
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# 👇 Crear la carpeta de subida si no existe
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [10]:
# Configuraciones para el archivo cargado
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Cargar el modelo previamente entrenado
model = load_model('/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/modelo_transfer.keras')  # Cambia esto por el nombre de tu modelo

# Función para verificar la extensión del archivo
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Endpoint principal
@app.route('/')
def home():
    return render_template('upload_form.html')

# Endpoint para subir imagen
@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400
    file = request.files['file']

    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        # Preprocesar la imagen para el modelo
        img = image.load_img(file_path, target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0


        # Hacer predicción
        prediction = model.predict(img_array)
        score = float(prediction[0][0])
        porcentaje = round(score * 100, 2)

        if score > 0.5:
            resultado = "Positivo: Posible neumonía"
            diagnostico = "Positivo - Posible neumonía"
            mensaje = "Por favor, consulta con tu médico lo antes posible para una evaluación completa."
        else:
            resultado = "Negativo - No se detecta neumonia"
            diagnostico = "Negativo - No se detecta neumonia"
            mensaje = ""

        return render_template("resultado.html", porcentaje=porcentaje, message=resultado, diagnostico=diagnostico, mensaje=mensaje)

    return jsonify({"error": "Archivo no permitido"}), 400

##Iniciar Servidor

In [11]:
#shutil.move('/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/templates/upload_form.html', '/content/templates/upload_form.html')

'/content/templates/upload_form.html'

In [12]:
#shutil.move('/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/templates/resultado.html','/content/templates/resultado.html')

'/content/templates/resultado.html'

In [13]:
!ls /content/templates

resultado.html	upload_form.html


In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9ac9-34-169-18-175.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-11-b54e7c4d82f8>", line 17, in home
    return render_template('upload_form.html')
           ^^^^^^^^^^^^^^^^^^^^

##Preparación para Github

In [18]:
!pip freeze > "/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/proyecto_neumonia/requirements.txt"

In [15]:
gitignore_content = """
__pycache__/
*.pyc
*.h5
*.pt
*.pkl
.DS_Store
.env
.ipynb_checkpoints/
"""

with open("/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/proyecto_neumonia/.gitignore", "w") as f:
    f.write(gitignore_content)

In [16]:
readme_content = """
# Detección de Neumonía con IA

Este proyecto utiliza una red neuronal convolucional (CNN) entrenada para predecir si una radiografía de tórax muestra signos de neumonía.

## 🧠 Modelo

- Entrenado en Google Colab
- Basado en una arquitectura CNN
- Entrenado con dataset de imágenes de tórax

## 🚀 Demo (API)

Incluye una API construida en Flask que permite hacer predicciones con una imagen subida por el usuario.

## 📁 Estructura del proyecto

proyecto_neumonia/
├── modelo_entrenamiento.ipynb
├── api_demo.ipynb
├── modelo/
│ └── modelo_neumonia.h5
├── requirements.txt
├── README.md
└── .gitignore

**Descripción de los archivos:**

- `modelo_entrenamiento.ipynb`: Entrenamiento del modelo.
- `api_demo.ipynb`: Interfaz de prueba con Flask.
- `modelo/modelo_neumonia.h5`: Modelo entrenado.
- `requirements.txt`: Dependencias del proyecto.
- `README.md`: Descripción del proyecto.
- `.gitignore`: Exclusiones de Git.

---

## 🔧 Instalación

Para instalar las dependencias, ejecuta:

```bash
pip install -r requirements.txt

🧪 Prueba
Copia el notebook api_demo.ipynb y ejecuta una celda para subir una imagen de rayos X. El modelo clasificará la imagen como "Neumonía" o "Normal".

👨‍💻 Autor
Proyecto desarrollado como parte de un proyecto educativo.

"""

In [ ]:
with open("/content/drive/MyDrive/Bootcamp - DS9/Modulo 7/proyecto_neumonia/README.md", "w") as f:
    f.write(readme_content)